# Data Transformation and feature evaluation for set 3

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from knee_stress_predict.config import raw_data_dir, processed_data_dir

This notebook prepare data for training time serial model.

## Step 1: Load data

In [25]:
data_set_name = "set_2"
file_path = Path.joinpath(processed_data_dir, data_set_name,  "test_out.csv")
result = pd.read_csv(file_path)
result = result.drop('Unnamed: 0', axis=1)
result

,Code,Patella_PN,Femur_PN,Tibia_PN,Patella_Car_PN,Femur_Car_PN,Tibia_M_Car_PN,Tibia_L_Car_PN,Patella_volume,Femur_volume,...,med_frame_231,med_frame_232,med_frame_233,med_frame_234,med_frame_235,med_frame_236,med_frame_237,med_frame_238,med_frame_239,med_frame_240
0,9905863M00,923,4011,3009,4745,19095,2840,2840,28699.200017,229006.314467,...,7.940522,8.114698,8.086311,8.015185,7.980772,7.937262,7.907591,7.878760,8.074722,7.987107
1,9905863M12,884,4110,2790,4585,19025,2840,2840,28197.827325,246641.112279,...,8.946852,8.747979,8.417266,8.344149,8.372133,8.370408,8.596495,8.629192,8.235795,8.145109
2,9908796M00,641,2861,2268,4120,20295,2840,2840,15222.439598,141352.763773,...,10.066787,10.434282,10.759167,11.088811,11.352691,11.744940,12.900606,12.137175,12.298610,12.747186
3,9908796M12,625,3022,2171,4715,19385,2840,2840,14931.893097,149215.886238,...,11.839959,12.119081,12.394084,12.853664,13.399544,14.055142,16.243515,13.738163,14.097919,14.264049
4,9912946M00,788,4137,2689,5575,21035,2840,2840,22692.085291,248976.459037,...,8.264437,8.744334,8.920675,8.164564,8.124532,7.814567,7.941459,8.397485,9.203511,8.622799
5,9912946M12,791,4298,2533,5685,20945,2840,2840,22884.798440,258760.708517,...,9.214134,9.436297,10.116617,11.398787,12.224302,10.590812,9.509462,9.449929,9.582999,8.858046
6,9954040M00,803,3417,2512,5015,20230,2840,2350,23008.003160,182608.626657,...,19.147572,15.116287,13.531854,15.764213,14.066882,17.350437,18.653992,13.406596,15.773174,18.946312
7,9954040M12,812,3233,2740,4885,20345,2840,2360,22994.325369,172181.530610,...,10.667857,10.417480,10.339953,11.016376,11.634765,11.229431,11.648008,11.479815,11.997889,12.955767
8,9958234M00,759,3696,2606,5235,21865,2840,2840,20235.363396,208302.746227,...,6.941909,6.842664,6.783639,6.708703,6.727243,6.913268,6.510352,6.397964,6.315529,6.462003
9,9958234M12,752,3674,2669,5225,22195,2840,2840,20204.268573,207124.042497,...,9.722022,10.054302,9.890096,9.805895,9.761503,9.837078,9.859464,10.412395,10.144455,10.161654


## Step 2: Transform to tall table

Now let's create a tall table, so we can extract data in a specific tall format, which is more useful for training models.

In [26]:
column_names = list(result.columns)
column_names = [item for item in column_names if 'med_frame' not in item]
column_names = [item for item in column_names if 'lat_frame' not in item]

result = pd.wide_to_long(result,
                    stubnames=["med_frame_", "lat_frame_"],
                    i=column_names,
                    j="frame")
result = result.rename(columns={"med_frame_": "Max_tib_med_contact_pressure", "lat_frame_": "Max_tib_lat_contact_pressure"})

In [27]:
result

Max_tib_med_contact_pressure  \
Code       Patella_PN Femur_PN Tibia_PN Patella_Car_PN Femur_Car_PN Tibia_M_Car_PN Tibia_L_Car_PN Patella_volume Femur_volume  Tibia_volume  Patella_Car_volume Femur_Car_volume Tibia_M_Car_volume Tibia_L_Car_volume Patella_bounds_x Femur_bounds_x Tibia_bounds_x Patella_Car_bounds_x Femur_Car_bounds_x Tibia_M_Car_bounds_x Tibia_L_Car_bounds_x Patella_bounds_y Femur_bounds_y Tibia_bounds_y Patella_Car_bounds_y Femur_Car_bounds_y Tibia_M_Car_bounds_y Tibia_L_Car_bounds_y Patella_bounds_z Femur_bounds_z Tibia_bounds_z Patella_Car_bounds_z Femur_Car_bounds_z Tibia_M_Car_bounds_z Tibia_L_Car_bounds_z Max_dist_femur_tibia_lat_car Min_dist_femur_tibia_lat_car Mean_dist_femur_tibia_lat_car Max_dist_femur_tibia_med_car Min_dist_femur_tibia_med_car Mean_dist_femur_tibia_med_car frame                                 
9905863M00 923        4011     3009     4745           19095        2840           2840           28699.200017   229006.314467 163757.709784 3762.152952        16928.419729     2255.521204        2705.548061        50.430611        85.637450      84.724161      49.454165            81.790594          30.589494            36.405833            48.693488        83.241612      63.233628      33.007412            48.066266          9.819381             13.747335            27.492164        73.671344      70.012669      17.505247            77.312018          51.821435            38.66946             17.745715                    2.039169                     6.118655                      12.363764                    1.604390                     4.737838                      0                          6.611789   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          1                          6.370533   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          2                          6.274528   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            